<a href="https://colab.research.google.com/github/DataSavvyYT/AI-engineering-course/blob/main/07_fine_tuning_llm/0_qlora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run this cell first to install required libraries
!pip install -q -U torch bitsandbytes transformers peft accelerate

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from huggingface_hub import login

In [ ]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')

In [ ]:
# 1. Login to Hugging Face (Required for Gemma)
# Replace 'YOUR_HF_TOKEN' with your actual token
login(token=HF_TOKEN)


In [ ]:
# 2. Configuration for 4-bit Quantization (QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,              # Enable 4-bit loading
    bnb_4bit_quant_type="nf4",      # Normalized Float 4 (optimized for LLMs)
    bnb_4bit_compute_dtype=torch.float16, # Compute in float16 for speed
    bnb_4bit_use_double_quant=True, # Quantize the quantization constants
)

In [ ]:
# 3. Load the Model & Tokenizer
model_id = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto" # Automatically maps to T4 GPU
)

In [ ]:
# 4. Prepare model for QLoRA training
# This freezes base weights and prepares layers for low-bit training
model = prepare_model_for_kbit_training(model)


In [ ]:
# 5. Define LoRA Config
# Gemma target modules usually include q_proj, k_proj, v_proj, o_proj, etc.
peft_config = LoraConfig(
    r=8,                            # Rank (lower = faster, less memory)
    lora_alpha=16,                  # Alpha scaling
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
# 6. Apply LoRA Adapters
model = get_peft_model(model, peft_config)

In [ ]:
# 7. Verify Trainable Parameters
model.print_trainable_parameters()

In [ ]:
# Test Inference (sanity check)
input_text = "Explain quantum physics in one sentence."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

In [ ]:
outputs = model.generate(**input_ids, max_new_tokens=50)
print("\nOutput:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))